## 1. Default Message

In [1]:
# Without helper

from gai.messages.typing import MessagePydantic, DefaultBodyPydantic
import time
message = MessagePydantic(
    **{
        "header": {
            "sender": "User",
            "recipient": "Assistant",
            "timestamp": time.time(),
        },
        "body": {
            "type": "default",
            "content": "Hello, how are you?",
        }
    }
)
print(message)
assert isinstance(message.body,DefaultBodyPydantic)

# With helper

from gai.messages import create_message, convert_to_chat_messages

user_message = create_message(role="user", content="What is the capital of France?")
print(user_message)
assistant_message = create_message(role="assistant", content="The capital of France is Paris.")
print(assistant_message)

assert user_message.header.sender == "User"
assert user_message.header.recipient == "Assistant"
assert user_message.body.content == "What is the capital of France?"

assert assistant_message.header.sender == "Assistant"
assert assistant_message.header.recipient == "User"
assert assistant_message.body.content == "The capital of France is Paris."

gai_messages = [user_message, assistant_message]
convert_to_chat_messages(gai_messages)


id='684537eb-39ba-46ed-9dcc-c7f5cc6774b0' header=MessageHeaderPydantic(sender='User', recipient='Assistant', timestamp=1749748673.905353, order=0) body=DefaultBodyPydantic(type='default', content='Hello, how are you?')
id='61a22f24-89a5-42ad-a920-9f1904b045e9' header=MessageHeaderPydantic(sender='User', recipient='Assistant', timestamp=1749748673.9067647, order=0) body=DefaultBodyPydantic(type='default', content='What is the capital of France?')
id='686dcbf6-0507-4c2c-8396-a3fd2bf48251' header=MessageHeaderPydantic(sender='Assistant', recipient='User', timestamp=1749748673.9071078, order=0) body=DefaultBodyPydantic(type='default', content='The capital of France is Paris.')


[{'role': 'user', 'content': 'What is the capital of France?'},
 {'role': 'assistant', 'content': 'The capital of France is Paris.'}]

## 2. SendMessagePydantic vs ReplyMessagePydantic


In [4]:
from gai.messages import create_user_send_message,create_assistant_reply_chunk, convert_to_chat_messages
user_message = create_user_send_message(
    recipient="Sara",
    content="Tell me a story about a brave knight.",
)
print(user_message)
assistant_message = create_assistant_reply_chunk(sender="Sara",recipient="User", chunk_no=0, chunk="<eom>", content="Once upon a time, a brave knight saved a kingdom from a dragon.")
print(assistant_message)

assert user_message.header.sender == "User"
assert user_message.header.recipient == "Sara"
assert user_message.body.content == "Tell me a story about a brave knight."

assert assistant_message.header.sender == "Sara"
assert assistant_message.header.recipient == "User"
assert assistant_message.body.content == "Once upon a time, a brave knight saved a kingdom from a dragon."

gai_messages = [user_message, assistant_message]
convert_to_chat_messages(gai_messages)



MessagePydantic(
    id='40aaf412-efee-4eeb-9c54-db0dda702836',
    header=MessageHeaderPydantic(sender='User', recipient='Sara', timestamp=1749179438.0017743, order=0),
    body=SendBodyPydantic(
        type='send',
        dialogue_id='00000000-0000-0000-0000-000000000000',
        message_no=6,
        message_id='00000000-0000-0000-0000-000000000000.6',
        content_type='text',
        content='Tell me a story about a brave knight.'
    )
)

MessagePydantic(
    id='cbefa02b-114e-439d-b4ef-839c1a04876a',
    header=MessageHeaderPydantic(sender='Sara', recipient='User', timestamp=1749179438.0215435, order=0),
    body=ReplyBodyPydantic(
        type='reply',
        dialogue_id='00000000-0000-0000-0000-000000000000',
        message_no=7,
        message_id='00000000-0000-0000-0000-000000000000.7',
        chunk_no=0,
        chunk='<eom>',
        content_type='text',
        content='Once upon a time, a brave knight saved a kingdom from a dragon.'
    )
)

[{'role': 'user', 'content': 'Tell me a story about a brave knight.'},
 {'role': 'assistant',
  'content': 'Once upon a time, a brave knight saved a kingdom from a dragon.'}]

## 3. Serialization and Deserialization

In [5]:
from gai.messages import create_user_send_message,create_assistant_reply_chunk, convert_to_chat_messages,json,unjson
from gai.messages.typing import ReplyBodyPydantic, SendBodyPydantic
user_message = create_user_send_message(
    recipient="Sara",
    content="Tell me a story about a brave knight.",
)
assistant_message = create_assistant_reply_chunk(sender="Sara",recipient="User", chunk_no=0, chunk="<eom>", content="Once upon a time, a brave knight saved a kingdom from a dragon.")
gai_messages = [user_message, assistant_message]

# Convert array to JSON
jsoned = json(gai_messages)
print(jsoned)

# Convert JSON back as array and should not lose resolution
gai_messages_again = unjson(jsoned)
print(gai_messages_again)
assert gai_messages_again[0].header.sender == "User"
assert gai_messages_again[0].header.recipient == "Sara"
assert isinstance(gai_messages_again[0].body, SendBodyPydantic)

assert gai_messages_again[1].header.sender == "Sara"
assert gai_messages_again[1].header.recipient == "User"
assert isinstance(gai_messages_again[1].body, ReplyBodyPydantic)



[
    {
        "id": "48b1e41a-b9fc-4e8f-bb3c-c559ad489a96",
        "header": {
            "sender": "User",
            "recipient": "Sara",
            "timestamp": 1749179438.0490005,
            "order": 0
        },
        "body": {
            "type": "send",
            "dialogue_id": "00000000-0000-0000-0000-000000000000",
            "message_no": 8,
            "message_id": "00000000-0000-0000-0000-000000000000.8",
            "content_type": "text",
            "content": "Tell me a story about a brave knight."
        }
    },
    {
        "id": "5db5f27c-9bf6-48bf-bd64-3d82edb4cb6a",
        "header": {
            "sender": "Sara",
            "recipient": "User",
            "timestamp": 1749179438.049156,
            "order": 0
        },
        "body": {
            "type": "reply",
            "dialogue_id": "00000000-0000-0000-0000-000000000000",
            "message_no": 9,
            "message_id": "00000000-0000-0000-0000-000000000000.9",
            "chunk_no": 0,
            "chunk": "<eom>",
            "content_type": "text",
            "content": "Once upon a time, a brave knight saved a kingdom from a dragon."
        }
    }
]

[
    MessagePydantic(
        id='48b1e41a-b9fc-4e8f-bb3c-c559ad489a96',
        header=MessageHeaderPydantic(sender='User', recipient='Sara', timestamp=1749179438.0490005, order=0),
        body=SendBodyPydantic(
            type='send',
            dialogue_id='00000000-0000-0000-0000-000000000000',
            message_no=10,
            message_id='00000000-0000-0000-0000-000000000000.10',
            content_type='text',
            content='Tell me a story about a brave knight.'
        )
    ),
    MessagePydantic(
        id='5db5f27c-9bf6-48bf-bd64-3d82edb4cb6a',
        header=MessageHeaderPydantic(sender='Sara', recipient='User', timestamp=1749179438.049156, order=0),
        body=ReplyBodyPydantic(
            type='reply',
            dialogue_id='00000000-0000-0000-0000-000000000000',
            message_no=11,
            message_id='00000000-0000-0000-0000-000000000000.11',
            chunk_no=0,
            chunk='<eom>',
            content_type='text',
            content='Once upon a time, a brave knight saved a kingdom from a dragon.'
        )
    )
]

---

## 2. Async Message Bus

### a) Quick Start - Publish to LLM

In [3]:
from gai.messages import (
    MessagePydantic, 
    AsyncMessageBus, 
    create_user_send_message,
    create_assistant_reply_chunk
)    
from rich import print

amb = AsyncMessageBus()

# 1. Start the background bus loop

await amb.start()
print("Bus is ready.")

# 2. Subscribe and handle send to LLM

async def handle_send(msg: MessagePydantic):
    
    # Forward user message to LLM
    
    from gai.llm.openai import OpenAI
    client = OpenAI(client_config={"model":"gpt-4o-mini"})
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":msg.body.content}],
        max_tokens=100
        )
    
    # Forward LLM response to user
    
    await amb.publish(
        create_assistant_reply_chunk(
            sender=msg.header.recipient,  
            recipient=msg.header.sender,
            chunk_no=0,
            chunk="<eom>",
            content=resp.choices[0].message.content,
        )
    )
await amb.subscribe("send", {"Alice": handle_send})
    
# 3. Subscribe and handle replies    

async def handle_reply(msg: MessagePydantic):
    print(f"Reply from {msg.header.sender}: [green]{msg.body.content}[/]")
await amb.subscribe("reply", {"User": handle_reply})

# 4. Publish a message to the bus

msg = create_user_send_message(content="Hello, what is your name?")
await amb.publish(msg)


Bus is ready.

Reply from Assistant: Hello! I'm known as Assistant. How can I help you today?


### b) Same Message Type - Multiple Subscribers

In [1]:

import asyncio
from gai.messages import MessagePydantic, AsyncMessageBus, create_user_send_message
from rich import print

amb = AsyncMessageBus()

# 1. Start the background bus loop
await amb.start()
print("Bus is ready.")

# 2. Create 2 handlers to show multiple subscriptions to the same message type

async def handler_1(msg: MessagePydantic):
    print(f"[bright_yellow] handler_1 > {msg}[/bright_yellow]")

async def handler_2(msg: MessagePydantic):
    print(f"[bright_green] handler_2 > {msg}[/bright_green]")

# 2. Subscribe the callback to the message type="send"

await amb.subscribe("send", {"Alice": handler_1})
await amb.subscribe("send", {"Bob":handler_2})
print("Subscribed to message type 'send'")

# 4. Confirm the bus is started and publish a message

if not amb.is_started:
    raise RuntimeError("Bus is not started, cannot publish message.")

msg = create_user_send_message(
    recipient="Assistant",
    content="Hello, what is your name?"
    )

print("Publishing message.")
await amb.publish(msg)
print("Message published.")
await asyncio.sleep(0.1)  # Give time for the message to be processed

# 5. Wait a little for message to be received before cancelling task
await amb.stop()

print("Bus stopped.")


Bus is ready.

Subscribed to message type 'alpha'

Publishing message.

Message published.

 handler_1 > id='3630a374-dda7-4805-b982-f691f0034ac0' header=MessageHeaderPydantic(sender='User', 
recipient='Assistant', timestamp=1749180766.9110599, order=0) body=SendBodyPydantic(type='send', 
dialogue_id='00000000-0000-0000-0000-000000000000', message_no=1, 
message_id='00000000-0000-0000-0000-000000000000.1', content_type='text', content='Hello, what is your name?')

 handler_2 > id='3630a374-dda7-4805-b982-f691f0034ac0' header=MessageHeaderPydantic(sender='User', 
recipient='Assistant', timestamp=1749180766.9110599, order=0) body=SendBodyPydantic(type='send', 
dialogue_id='00000000-0000-0000-0000-000000000000', message_no=1, 
message_id='00000000-0000-0000-0000-000000000000.1', content_type='text', content='Hello, what is your name?')

Bus stopped.

### b) Multiple Message Type - One Subscriber

In [5]:
import asyncio
from gai.messages import (
    MessagePydantic, 
    AsyncMessageBus, 
    create_user_send_message,
    create_assistant_reply_chunk
    )
from rich import print

amb = AsyncMessageBus()

# 1. Start the background bus loop AND wait for signal to be ready
await amb.start()
print("Bus is ready.")

# 2. Create 2 handlers: one for SendMessagePydantic and one for ReplyMessagePydantic

async def send_handler(msg: MessagePydantic):
    print(f"[bold bright_yellow]Send Message Received: {msg}[/bold bright_yellow]")

async def reply_handler(msg: MessagePydantic):
    print(f"[bold bright_green]Send Message Received: {msg}[/bold bright_green]")

# 3. Subscribe the callback to the message type="alpha"

await amb.subscribe("send", {"Alice":send_handler})
await amb.subscribe("reply", {"Alice":reply_handler})
print("Subscribed to send and reply message types")

# 4. Confirm the bus is started and publish a message

if not amb.is_started:
    raise RuntimeError("Bus is not started, cannot publish message.")

send_msg = create_user_send_message(content="Tell me a one sentence story.")

print("Publishing send message.")
await amb.publish(send_msg)
print("Send message published.")

reply_msg = create_assistant_reply_chunk(sender="Sara",chunk_no=0, chunk="<eom>", content="Once upon a time, a brave knight saved a kingdom from a dragon.")
print("Publishing reply message.")
await amb.publish(reply_msg)
print("Reply message published.")

# 5. Wait a little for message to be received before cancelling task

await asyncio.sleep(0.1)

# 6. Stop the bus
await amb.stop()
print("Bus stopped.")


Bus is ready.

Subscribed to send and reply message types

Publishing send message.

Send message published.

Publishing reply message.

Reply message published.

Send Message Received: id='c590e19b-dea6-4fdb-8271-73368580756a' header=MessageHeaderPydantic(sender='User', 
recipient='Assistant', timestamp=1749181012.3732293, order=0) body=SendBodyPydantic(type='send', 
dialogue_id='00000000-0000-0000-0000-000000000000', message_no=4, 
message_id='00000000-0000-0000-0000-000000000000.4', content_type='text', content='Tell me a one sentence story.')

Send Message Received: id='a84401e7-1ced-4070-a677-551bf10b6910' header=MessageHeaderPydantic(sender='Sara', 
recipient='User', timestamp=1749181012.3782616, order=0) body=ReplyBodyPydantic(type='reply', 
dialogue_id='00000000-0000-0000-0000-000000000000', message_no=5, 
message_id='00000000-0000-0000-0000-000000000000.5', chunk_no=0, chunk='<eom>', content_type='text', content='Once 
upon a time, a brave knight saved a kingdom from a dragon.')

Bus stopped.

### c) Same Message Type - Same Handler - Different Handler State

In [1]:
import asyncio
from gai.messages import (
    MessagePydantic,
    AsyncMessageBus,
    create_assistant_reply_content,
    create_user_send_message
)
    
from rich import print

# 3. Start the background bus loop AND wait for signal to be ready

amb = AsyncMessageBus()
await amb.start()
print("Bus is ready.")

# 1. Create 2 assistant send_handlers to show multiple subscriptions to the same message type

class Host:
    def __init__(self,name:str):
        self.name = name
        
    async def send_handler(self, msg: MessagePydantic):
        print(f"[bold bright_yellow]{self.name} > {msg}[/bold bright_yellow]")
        await amb.publish(
            create_assistant_reply_content(
                sender=self.name,
                recipient="User",
                content=f"My name is {self.name}."                            
            )
        )
        
host_1 = Host("Host 1")
host_2 = Host("Host 2")

# 2. Create 1 user reply_handlers to handle replies

async def reply_handler(msg: MessagePydantic):
    print(f"[bold bright_green]User > {msg}[/bold bright_green]")


# 2. Subscribe the callback to the message type="alpha" but same handler belonging to different objects

await amb.subscribe("send", {"Alice":host_1.send_handler})
await amb.subscribe("send", {"Bob":host_2.send_handler})
await amb.subscribe("reply", {"User":reply_handler})

print("Subscribed to message type 'send'")

# 4. Confirm the bus is started and publish a message

if not amb.is_started:
    raise RuntimeError("Bus is not started, cannot publish message.")

msg = create_user_send_message(
    recipient="*",
    content="Hello, what is your name?"
)
print("Publishing message.")
await amb.publish(msg)
print("Message published.")

# 5. Wait a little for message to be received before cancelling task

await asyncio.sleep(0.1)
status=await amb.stop()
print("Bus stopped.")


Bus is ready.

Subscribed to message type 'send'

Publishing message.

Message published.

Host 1 > id='5f5fdb1e-eaf9-46f8-876c-19f8279a9e37' header=MessageHeaderPydantic(sender='User', recipient='*', 
timestamp=1749181354.8389497, order=0) body=SendBodyPydantic(type='send', 
dialogue_id='00000000-0000-0000-0000-000000000000', message_no=1, 
message_id='00000000-0000-0000-0000-000000000000.1', content_type='text', content='Hello, what is your name?')

Host 2 > id='5f5fdb1e-eaf9-46f8-876c-19f8279a9e37' header=MessageHeaderPydantic(sender='User', recipient='*', 
timestamp=1749181354.8389497, order=0) body=SendBodyPydantic(type='send', 
dialogue_id='00000000-0000-0000-0000-000000000000', message_no=1, 
message_id='00000000-0000-0000-0000-000000000000.1', content_type='text', content='Hello, what is your name?')

User > id='4b57a61a-0eb4-4864-9e2c-3a9a4097fa04' header=MessageHeaderPydantic(sender='Host 1', recipient='User', 
timestamp=1749181354.8443098, order=0) body=ReplyBodyPydantic(type='reply', 
dialogue_id='00000000-0000-0000-0000-000000000000', message_no=2, 
message_id='00000000-0000-0000-0000-000000000000.2', chunk_no=0, chunk='<eom>', content_type='text', content='My 
name is Host 1.')

User > id='f12e6700-692a-4433-b59d-387b0178329c' header=MessageHeaderPydantic(sender='Host 2', recipient='User', 
timestamp=1749181354.8464136, order=0) body=ReplyBodyPydantic(type='reply', 
dialogue_id='00000000-0000-0000-0000-000000000000', message_no=3, 
message_id='00000000-0000-0000-0000-000000000000.3', chunk_no=0, chunk='<eom>', content_type='text', content='My 
name is Host 2.')

Bus stopped.